In [236]:
from langchain.text_splitter import HTMLHeaderTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
import numpy as np
import string

In [119]:
with open('../san_francisco-ca-2.html', encoding='latin-1') as f:
    planning_code = f.read()

In [120]:
len(planning_code)

36203998

In [142]:
def remove_css_from_html(html_content):
    # Credit to gpt4
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove all CSS style attributes
    for tag in soup.recursiveChildGenerator():
        if hasattr(tag, 'attrs'):
            tag.attrs = {key: value for key, value in tag.attrs.items() if key != 'style'}

    # Remove all CSS class attributes
    for tag in soup.find_all(True, {'class': True}):
        if tag.name == 'div' and 'ChapAn' in tag.get('class', []):
            continue  # Skip this tag if it's a <div> with class "ChapAn"
        del tag['class']

    # Find and remove all AnnotationDrawer elements
    for tag in soup.find_all('annotationdrawer'):
        tag.decompose()
        
    return str(soup)

# Remove CSS from the HTML
clean_html = remove_css_from_html(planning_code)

# Write the cleaned HTML to a new file
with open('cleaned_html_file.html', 'w') as file:
    file.write(clean_html)



In [143]:
len(clean_html) / len(planning_code)

0.41231103813451764

In [198]:
indentations = ['>' + '\xa0'*i + '(' for i in range(1, 20) ]

In [239]:
little_letters = ['(' + l + ')' for l in string.ascii_lowercase]

In [252]:
numbered_sections = ['(' + str(d) + ')' for d in range(100)]

In [253]:
splitter = RecursiveCharacterTextSplitter(
    ['SEC'] + little_letters + numbered_sections + ['\n\n', '\n', ' '],
    #chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
    add_start_index = False
)

In [254]:
chunks = splitter.split_text(clean_html)
len(chunks)

7735

In [256]:
np.random.seed(1)
np.random.choice(chunks, 3)

array(['(c)(6).</span></div></td></tr><tr><td><div><span>Dwelling Unit Density</span></div></td><td><div><span>§§ <link to="{{ pathname: \'/codes/san_francisco/latest/sf_planning/0-0-0-17783\', hash: \'#JD_102\' }}"/>102, <link to="{{ pathname: \'/codes/san_francisco/latest/sf_planning/0-0-0-19952\', hash: \'#JD_207\' }}"/>207</span></div></td><td colspan="3"><div><span>1 unit per 800 square foot lot area, or the density permitted in the nearest R District, whichever is greater.</span></div></td></tr><tr><td><div><span>Group Housing Density</span></div></td><td><div><span>§ <link to="{{ pathname: \'/codes/san_francisco/latest/sf_planning/0-0-0-20056\', hash: \'#JD_208\' }}"/>208</span></div></td><td colspan="3"><div><span>1 bedroom per 275 square foot lot area, or the density permitted in the nearest R District, whichever is greater.</span></div></td></tr><tr><td><div><span>Homeless Shelter Density</span></div></td><td><div><span>§§ <link to="{{ pathname: \'/codes/san_francisco/latest/